# TODO

 - !!! figure out which entropy to use for `a(o)`
 - which object attributes for computation of typicality?

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


from ngrams import Ngram

In [ ]:
df = pd.read_csv("../NMvW_data/v0.csv.gz", 
                 dtype=dict(Provenance="string", RelatedWorks="string"))

# TODO: save & load DF s.t. these lines are not necessary here                
df["ObjectID"] = df.ObjectID.astype("int")
df = df.set_index("ObjectID")

In [ ]:
docs = list(df.Title.dropna())
ng = Ngram(ns=2, documents=docs, precompute_freqs=True)

In [ ]:
log_probs = [np.sum([ng.cond_prob(w, *rest, log=True) for *rest, w in list(ng.iter_ngrams(s, as_tuples=True))])
             for s in tqdm(docs)]

lens = [len(list(ng.iter_ngrams(s, as_tuples=True))) for s in docs[:1000]]

typs = [abs(H - (-lgp/l)) for lgp, l in zip(log_probs, lens)]

In [ ]:
log_P_o = np.asarray(log_probs)
-np.sum(np.exp(P_o)*P_o), H

In [ ]:
_ = plt.hist(typs, bins=50)

In [ ]:
lens = [len(list(ng.iter_ngrams(s, as_tuples=True))) for s in docs[:1000]]

_=plt.hist(lens)

In [ ]:
mean_logs = [lp/l for lp, l in zip(log_probs, lens)]

_ = plt.hist(mean_logs, bins=50)

In [ ]:
plt.plot(lens, mean_logs, ".")

In [ ]:
sorted(zip(log_probs, docs), reverse=True)[:10]

# VOCAB

In [ ]:
voc_probs = {gram: ng.prob(*gram.split()) for gram in tqdm(ng.vocab(2))}

In [ ]:
P = np.asarray(list(voc_probs.values()))

H = -np.sum(P*np.log2(P))

In [ ]:
sorted(voc_probs.items(), key=lambda t: t[1], reverse=False)[:10]

In [ ]:
plt.loglog(sorted(voc_probs.values(), reverse=True), ".")

In [ ]:
_=plt.hist(-np.log2(P), bins=100)
_=plt.axvline(H, c="r")
# plt.xscale("log")
# plt.yscale("log")

In [ ]:
sorted(voc_probs.items(), key=lambda t: abs(H-(-np.log2(t[1]))), reverse=False)

In [ ]:
np.log2(ng.prob('merbaboe')*ng.prob('</s>')) - np.log2(ng.prob('merbaboe </s>'))

---
# Prep for Server

need to:

  1. load DataFrame
  2. get text -> put in Dataset class
  3. construct NGram(n) class
  4. compute typicalities  
    1. a(o) 
    2. a(w) for each w in o
    3. -> {o: (a, {w: a_w})}
  5. 


parameters:

  - n for NGram class
  - 

In [2]:
# 1. 
df = pd.read_csv("../NMvW_data/v0.csv.gz", 
                 dtype=dict(Provenance="string", RelatedWorks="string"))

# TODO: save & load DF s.t. these lines are not necessary here                
df["ObjectID"] = df.ObjectID.astype("int")
df = df.set_index("ObjectID")

df = df.replace(np.nan, "")

# 2.
# def get_text(row):
#     return row[["Title", "Description"]]    
    
# texts = (t for i, row in df.iterrows() for t in get_text(row))

texts = list(df["Title"]) + list(df["Description"])

ng = Ngram(ns=3, documents=texts, precompute_freqs=True)

(1, 2)-grams: Padding documents...: 100%|██████████| 1159854/1159854 [00:00<00:00, 1863269.40it/s]


(1, 2)-grams: Term Document Matrix constructed...
(1, 2)-grams: Term frequencies precomputed...
(1, 2)-grams: Init done


In [19]:
for i, row in df[:100].iterrows():
    o_prob = 1.
    for t in row[["Title", "Description"]]:
        print(t)
        grams = list(ng.iter_ngrams(t, as_tuples=True))
        
        for *rest, w in grams:
            w_prob = ng.cond_prob(w, *rest)
            print(w, "|", rest, w_prob)
            o_prob *= w_prob
        
    print("P(o): ", o_prob)
    print("\n\n")

BESCHILDERDE EN GEVLOCHTEN HOED VAN PLANTAARDIGE VEZEL
beschilderde | ['<s>'] 0.0006621523053763663
en | ['beschilderde'] 0.0283178360101437
gevlochten | ['en'] 0.0002935505021119816
hoed | ['gevlochten'] 0.010371808091223388
van | ['hoed'] 0.10660847880299251
plantaardige | ['van'] 0.00029300159042151163
vezel | ['plantaardige'] 0.15863453815261044
</s> | ['vezel'] 0.21833333333333332

</s> | ['<s>'] 0.3443148879083057
P(o):  2.1266233473501034e-17



MANDJE
mandje | ['<s>'] 0.00018967904581093827
</s> | ['mandje'] 0.13957759412304868

</s> | ['<s>'] 0.3443148879083057
P(o):  9.115717675239702e-06



MANDJE VAN GEVLOCHTEN BAMBOE
mandje | ['<s>'] 0.00018967904581093827
van | ['mandje'] 0.09641873278236915
gevlochten | ['van'] 0.003078736408444217
bamboe | ['gevlochten'] 0.023169770121914235
</s> | ['bamboe'] 0.09326493028150487

</s> | ['<s>'] 0.3443148879083057
P(o):  4.189374449784718e-11



GEVLOCHTEN RIJSTZAK
gevlochten | ['<s>'] 0.002576186313104925
rijstzak | ['gevlochten'] 0.000

WAYANGPOP
wayangpop | ['<s>'] 8.449339313396342e-05
</s> | ['wayangpop'] 0.1092896174863388
Herk: Onbekend
herk | ['<s>'] 0.011792863584554608
onbekend | ['herk'] 0.004924324715765081
</s> | ['onbekend'] 0.18843683083511778
P(o):  1.010493226070871e-10



HUISVERSIERING IN DE VORM VAN EEN SINGAKOP
huisversiering | ['<s>'] 3.5349276719311226e-05
in | ['huisversiering'] 0.1111111111111111
de | ['in'] 0.2393476735001444
vorm | ['de'] 0.014820909714474122
van | ['vorm'] 0.6396225848132456
een | ['van'] 0.13110600331402555
singakop | ['een'] 5.681608349691631e-06
</s> | ['singakop'] 0.3076923076923077

</s> | ['<s>'] 0.3443148879083057
P(o):  7.032863643113811e-16



BLIKKEN KEUKENLEPELTJE
blikken | ['<s>'] 0.00025520453436380785
keukenlepeltje | ['blikken'] 0.0012004801920768306
</s> | ['keukenlepeltje'] 1.0

</s> | ['<s>'] 0.3443148879083057
P(o):  1.0548705959563727e-07



RONDE GEVLOCHTEN WAN
ronde | ['<s>'] 0.0008216551393537462
gevlochten | ['ronde'] 0.004290560766314109
wan | ['gevlo

In [87]:
def is_boundary_gram(gram):
    return (gram.find("<s>") >= 0) or (gram.find("</s>") >= 0)

def process_object(row):
    obj_prob = 0.
    
    l = 0
    for text in row:
        grams = list(ng.iter_ngrams(text, as_tuples=True))
        for *rest, w in grams:
            w_prob = ng.cond_prob(w, *rest, log=True)
            obj_prob += w_prob
            l += 1
    
            if not is_boundary_gram(" ".join((*rest, w))):
                yield (*rest, w), w_prob
    yield obj_prob/l

In [88]:
*w_probs, obj_prob = list(process_object(df[["Title", "Description"]].iloc[10000]))

dict(w_probs), obj_prob

({('traditionele', 'wapens'): -5.0275493352477,
  ('wapens', 'in'): -3.8189231264811436,
  ('in', 'de'): -2.062820312001257,
  ('de', 'melanesische'): -12.345948978679282,
  ('melanesische', 'oorlogsvoering'): -0.7903618511442989,
  ('oorlogsvoering', 'waren'): -2.141745188079611,
  ('waren', 'pijl'): -5.8199264020354775,
  ('pijl', 'en'): -1.5093046738019105,
  ('en', 'boog'): -7.21543283314986,
  ('boog', 'speren'): -4.642111697372458,
  ('speren', 'en'): -2.0233461103744723,
  ('en', 'knotsen'): -10.687355395970354,
  ('knotsen', 'vooral'): -3.3146634215147337,
  ('vooral', 'pijl'): -5.899120636800302,
  ('boog', 'werden'): -4.769625120706724,
  ('werden', 'gebruikt'): -4.896320812394821,
  ('gebruikt', 'in'): -3.949504160923775,
  ('de', 'strijd'): -9.716536525667879,
  ('strijd', 'in'): -3.209875597242389,
  ('in', 'tegenstelling'): -8.302597536369317,
  ('tegenstelling', 'tot'): -0.12086232174843659,
  ('tot', 'polynesie'): -7.68558948691895,
  ('polynesie', 'waar'): -1.163975735

In [100]:
result = [process_object(row)
          for i, row in tqdm(df[["Title", "Description"]].iterrows(),
                            total=len(df))]





# results = [(p, dict(w_probs)) for *w_probs, p in 
#            tqdm(df[["Title", "Description"]].apply(process_object, axis=0), 
#                                                     total=len(df))]

100%|██████████| 579927/579927 [01:12<00:00, 8053.87it/s] 


In [92]:
df[["Title", "Description"]].apply(len, axis="columns")

ObjectID
1          2
2          2
3          2
4          2
5          2
          ..
1140716    2
1140717    2
1140718    2
1140719    2
1140720    2
Length: 579927, dtype: int64

In [13]:
probs = {s: np.sum([ng.cond_prob(w, *rest) 
                    for *rest, w in list(ng.iter_ngrams(s, as_tuples=True))])
        for s in tqdm(texts)}

100%|██████████| 1159854/1159854 [05:26<00:00, 3553.05it/s] 


In [79]:
sorted(probs.items(), key=lambda tup: tup[1], reverse=False)[:10]

[('To', 0.0001982575590004394),
 ('ALLERLEI', 0.0002878985384509211),
 ('AT', 0.0004922072631985176),
 ('OM', 0.0007258515391679908),
 ('GEEN', 0.0008908656651382611),
 ('geen', 0.0008908656651382611),
 ('(Geen)', 0.0008908656651382611),
 ('DOOR 2', 0.0011457629470313724),
 ('DOOR 3', 0.0011457629470313724),
 ('DOOR 1', 0.0011457629470313724)]